In [1]:
import numpy as np 
import pandas as pd
import matplotlib as mtl

In [82]:
%%time
item_cat = pd.read_csv("raw_data/item_categories.csv")
items = pd.read_csv("raw_data/items.csv")
sales_train = pd.read_csv("raw_data/sales_train.csv")
shops = pd.read_csv("raw_data/shops.csv")
test = pd.read_csv("raw_data/test.csv")
# sample_submission = pd.read_csv("sample_submission.csv")

CPU times: user 866 ms, sys: 63.3 ms, total: 930 ms
Wall time: 926 ms


# Item categories #

In [4]:
item_cat.sample(10)
# clean description

,item_category_name,item_category_id
13,Игровые консоли - PSP,13
43,Книги - Аудиокниги,43
0,PC - Гарнитуры/Наушники,0
11,Игровые консоли - PS3,11
3,Аксессуары - PS4,3
59,Музыка - Музыкальное видео,59
79,Служебные,79
14,Игровые консоли - PSVita,14
42,"Книги - Артбуки, энциклопедии",42
76,Программы - Для дома и офиса (Цифра),76


In [5]:
item_cat.info()
# 0 null values

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84 entries, 0 to 83
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   item_category_name  84 non-null     object
 1   item_category_id    84 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 1.4+ KB


In [6]:
item_cat.describe(include=[object])
# no duplicates

,item_category_name
count,84
unique,84
top,PC - Гарнитуры/Наушники
freq,1


In [58]:
item_cat.describe()
# normal indexing range

,item_category_id
count,84.000000
mean,41.500000
std,24.392622
min,0.000000
25%,20.750000
50%,41.500000
75%,62.250000
max,83.000000


# Shops 

In [43]:
shops.head(10)

,shop_name,shop_id
0,"!Якутск Орджоникидзе, 56 фран",0
1,"!Якутск ТЦ ""Центральный"" фран",1
2,"Адыгея ТЦ ""Мега""",2
3,"Балашиха ТРК ""Октябрь-Киномир""",3
4,"Волжский ТЦ ""Волга Молл""",4
5,"Вологда ТРЦ ""Мармелад""",5
6,"Воронеж (Плехановская, 13)",6
7,"Воронеж ТРЦ ""Максимир""",7
8,"Воронеж ТРЦ Сити-Парк ""Град""",8
9,Выездная Торговля,9


In [83]:
shops['shop_name'] = shops['shop_name'].str.replace("!|\?|²|\*|/", '', regex=True)
shops.head(10)
# cleaning shop names from excess symbols

,shop_name,shop_id
0,"Якутск Орджоникидзе, 56 фран",0
1,"Якутск ТЦ ""Центральный"" фран",1
2,"Адыгея ТЦ ""Мега""",2
3,"Балашиха ТРК ""Октябрь-Киномир""",3
4,"Волжский ТЦ ""Волга Молл""",4
5,"Вологда ТРЦ ""Мармелад""",5
6,"Воронеж (Плехановская, 13)",6
7,"Воронеж ТРЦ ""Максимир""",7
8,"Воронеж ТРЦ Сити-Парк ""Град""",8
9,Выездная Торговля,9


In [45]:
shops.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   shop_name  60 non-null     object
 1   shop_id    60 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 1.1+ KB


In [61]:
shops.describe(include=[object])

,shop_name
count,60
unique,59
top,Жуковский ул. Чкалова 39м
freq,2


In [47]:
shops.loc[shops.duplicated("shop_name", keep=False)]

,shop_name,shop_id
10,Жуковский ул. Чкалова 39м,10
11,Жуковский ул. Чкалова 39м,11


In [30]:
test.loc[test.shop_id == 10]

,ID,shop_id,item_id
30600,30600,10,5037
30601,30601,10,5320
30602,30602,10,5233
30603,30603,10,5232
30604,30604,10,5268
...,...,...,...
35695,35695,10,18454
35696,35696,10,16188
35697,35697,10,15757
35698,35698,10,19648


In [31]:
test.loc[test.shop_id == 11]

,ID,shop_id,item_id


In [32]:
sales_train.loc[sales_train.shop_id == 10]

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
53564,26.01.2013,0,10,6000,190.0,1.0
53565,12.01.2013,0,10,6007,180.0,1.0
53566,08.01.2013,0,10,6093,200.0,1.0
53567,12.01.2013,0,10,6093,200.0,1.0
53568,13.01.2013,0,10,6095,1321.0,1.0
...,...,...,...,...,...,...
2919918,03.10.2015,33,10,7893,2465.0,1.0
2919919,03.10.2015,33,10,7879,2209.0,1.0
2919920,20.10.2015,33,10,7942,3418.0,1.0
2919921,24.10.2015,33,10,7933,822.0,1.0


In [33]:
sales_train.loc[sales_train.shop_id == 11]

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
2461045,27.02.2015,25,11,22162,237.11,1.0
2461046,23.02.2015,25,11,22162,237.11,1.0
2461047,20.02.2015,25,11,22162,270.00,1.0
2461048,11.02.2015,25,11,22162,270.00,1.0
2461049,10.02.2015,25,11,22162,270.00,1.0
...,...,...,...,...,...,...
2462003,12.02.2015,25,11,10389,219.00,1.0
2462004,07.02.2015,25,11,10382,460.00,1.0
2462005,17.02.2015,25,11,10379,195.00,1.0
2462006,18.02.2015,25,11,10298,99.00,1.0


In [84]:
# Dealing with duplicates of shop_name depending on their occurrence in test set.
# Better explained in the following part.
dup = shops.loc[shops.duplicated("shop_name", keep=False)] \
    .groupby("shop_name") \
    .agg(lambda x: list(x)) \
    .shop_id
test_shops = test.shop_id.to_numpy()
for idx, value in dup.items():
    value: np.ndarray = np.array(value)
    mask = np.isin(value, test_shops)
    if np.sum(mask) < 2:
        shops.loc[value, 'shop_id'] = value[~mask][0] if value[~mask].size > 0 else value[0]
        shops.drop_duplicates("shop_id", inplace=True)
        sales_train.loc[sales_train.shop_id.isin(value), 'shop_id'] = value[~mask][0] if value[~mask].size > 0 \
            else value[0]
shops.reset_index(drop=True, inplace=True)

In [49]:
shops.loc[shops.duplicated("shop_name")]

,shop_name,shop_id


In [85]:
shops.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59 entries, 0 to 59
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   shop_name  59 non-null     object
 1   shop_id    59 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 1.4+ KB


In [101]:
sales_train.loc[sales_train.shop_id == 11]

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day


# Items #

In [131]:
 items.head(20)

,item_name,item_id,item_category_id
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,***КОРОБКА (СТЕКЛО) D,4,40
5,***НОВЫЕ АМЕРИКАНСКИЕ ГРАФФИТИ (UNI) ...,5,40
6,***УДАР ПО ВОРОТАМ (UNI) D,6,40
7,***УДАР ПО ВОРОТАМ-2 (UNI) D,7,40
8,***ЧАЙ С МУССОЛИНИ D,8,40
9,***ШУГАРЛЭНДСКИЙ ЭКСПРЕСС (UNI) D,9,40


In [148]:
items['item_name'] = items['item_name'].str.replace("!|\?|²|\*|/", '', regex=True)
items.head(20)
# cleaning item names from excess symbols

,item_name,item_id,item_category_id
0,ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,ABBYY FineReader 12 Professional Edition Full ...,1,76
2,В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,КОРОБКА (СТЕКЛО) D,4,40
5,НОВЫЕ АМЕРИКАНСКИЕ ГРАФФИТИ (UNI) D,5,40
6,УДАР ПО ВОРОТАМ (UNI) D,6,40
7,УДАР ПО ВОРОТАМ-2 (UNI) D,7,40
8,ЧАЙ С МУССОЛИНИ D,8,40
9,ШУГАРЛЭНДСКИЙ ЭКСПРЕСС (UNI) D,9,40


In [149]:
items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22170 entries, 0 to 22169
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   item_name         22170 non-null  object
 1   item_id           22170 non-null  int64 
 2   item_category_id  22170 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 519.7+ KB


In [150]:
items.describe(include=[object])

,item_name
count,22170
unique,22166
top,МУЖИКИ (регион)
freq,2


In [151]:
items.loc[items.duplicated('item_name')]

,item_name,item_id,item_category_id
13012,КТО Я,13012,40
14690,МИХЕЙ И ДЖУМАНДЖИ Сука любовь,14690,55
14856,МУЖИКИ (регион),14856,40
15539,НА КРЮЧКЕ (BD),15539,37


In [ ]:
# Dealing with item duplicates which have different index and same name

In [152]:
# group togheter duplicates of same item name
dup = items.loc[items.duplicated("item_name", keep=False)]\
                            .groupby("item_name")\
                            .agg(lambda x: list(x))\
                            .item_id
dup                                                    

item_name
КТО Я                             [13011, 13012]
МИХЕЙ И ДЖУМАНДЖИ  Сука любовь       [12, 14690]
МУЖИКИ (регион)                   [14855, 14856]
НА КРЮЧКЕ (BD)                    [15537, 15539]
Name: item_id, dtype: object

In [153]:
test_items = test.item_id.to_numpy()
test_items

array([ 5037,  5320,  5233, ..., 15757, 19648,   969])

In [154]:
# for every duplicate name check if it take part of test set.
# If at least 2 duplicates of one name are in test set, we don't
# touch them. Otherwise, we merge all duplicates of the same name 
# in sales_train and items.
for idx, value in dup.items():
    value = np.array(value)
    mask = np.isin(value,test_items)
    if np.sum(mask) < 2:
        items.loc[value, 'item_id'] = value[~mask][0] if value[~mask].size > 0 else value[0]
        items.drop_duplicates("item_id", inplace = True)
        sales_train.loc[sales_train.item_id.isin(value), 'item_id'] = value[~mask][0] if value[~mask].size > 0 else value[0]
items.reset_index(inplace=True)

In [155]:
items.describe(include=['object'])
# ✨ clean ✨

,item_name
count,22166
unique,22166
top,ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D
freq,1


In [156]:
items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22166 entries, 0 to 22165
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   index             22166 non-null  int64 
 1   item_name         22166 non-null  object
 2   item_id           22166 non-null  int64 
 3   item_category_id  22166 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 692.8+ KB


# Sales

In [158]:
sales_train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [159]:
# deleting column date_block_num because it's redundant and 
# can be retrieved from date column.
sales_train.drop(columns='date_block_num',inplace=True)
sales_train.head()

,date,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,59,22154,999.00,1.0
1,03.01.2013,25,2552,899.00,1.0
2,05.01.2013,25,2552,899.00,-1.0
3,06.01.2013,25,2554,1709.05,1.0
4,15.01.2013,25,2555,1099.00,1.0


In [160]:
# Converting date to appropriate format
sales_train['date'] = pd.to_datetime(sales_train['date'], dayfirst=True)
sales_train.head()

,date,shop_id,item_id,item_price,item_cnt_day
0,2013-01-02,59,22154,999.00,1.0
1,2013-01-03,25,2552,899.00,1.0
2,2013-01-05,25,2552,899.00,-1.0
3,2013-01-06,25,2554,1709.05,1.0
4,2013-01-15,25,2555,1099.00,1.0


In [163]:
sales_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2935843 entries, 0 to 2935842
Data columns (total 5 columns):
 #   Column        Dtype         
---  ------        -----         
 0   date          datetime64[ns]
 1   shop_id       int64         
 2   item_id       int64         
 3   item_price    float64       
 4   item_cnt_day  float64       
dtypes: datetime64[ns](1), float64(2), int64(2)
memory usage: 112.0 MB


In [164]:
sales_train.drop_duplicates(inplace=True, ignore_index = True)
sales_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2935843 entries, 0 to 2935842
Data columns (total 5 columns):
 #   Column        Dtype         
---  ------        -----         
 0   date          datetime64[ns]
 1   shop_id       int64         
 2   item_id       int64         
 3   item_price    float64       
 4   item_cnt_day  float64       
dtypes: datetime64[ns](1), float64(2), int64(2)
memory usage: 112.0 MB


In [165]:
sales_train.isnull().sum()

date            0
shop_id         0
item_id         0
item_price      0
item_cnt_day    0
dtype: int64

In [166]:
sales_train.sample(10)

,date,shop_id,item_id,item_price,item_cnt_day
1652070,2014-05-02,4,4248,1049.00,1.0
1471914,2014-03-21,43,9863,629.00,1.0
1345587,2014-01-02,6,9313,349.00,1.0
1493941,2014-03-09,25,20784,1099.00,1.0
440921,2013-05-23,27,17156,149.00,1.0
744851,2013-08-28,42,14024,149.00,1.0
888845,2013-09-01,31,21893,149.00,1.0
830968,2013-08-11,14,9587,290.51,1.0
2086292,2014-10-10,53,6675,20990.00,1.0
1213949,2013-12-27,58,3076,799.00,1.0


In [167]:
sales_train.describe()

,shop_id,item_id,item_price,item_cnt_day
count,2.935843e+06,2.935843e+06,2.935843e+06,2.935843e+06
mean,3.300171e+01,1.019509e+04,8.908535e+02,1.242641e+00
std,1.622698e+01,6.325253e+03,1.729801e+03,2.618837e+00
min,0.000000e+00,0.000000e+00,-1.000000e+00,-2.200000e+01
25%,2.200000e+01,4.475000e+03,2.490000e+02,1.000000e+00
50%,3.100000e+01,9.336000e+03,3.990000e+02,1.000000e+00
75%,4.700000e+01,1.568400e+04,9.990000e+02,1.000000e+00
max,5.900000e+01,2.216900e+04,3.079800e+05,2.169000e+03


In [168]:
# drop examples:
# 1. which item_id doesn't belong to items
# 2. which shop_id doesn't belong to shops
# 3. which item_price is to low or to high
# 4. which item_cnt_day is to low or to high
drop_conditions = (~sales_train.item_id.isin(items['item_id'])) | \
                      (~sales_train.shop_id.isin(shops['shop_id'])) | \
                      (sales_train.item_price < 0.1) | \
                      (sales_train.item_price > 1e7) | \
                      (sales_train.item_cnt_day > 1e5) | \
                      (sales_train.item_cnt_day < -1e3)

sales_train = sales_train.drop(sales_train.loc[drop_conditions].index)
sales_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2935838 entries, 0 to 2935842
Data columns (total 5 columns):
 #   Column        Dtype         
---  ------        -----         
 0   date          datetime64[ns]
 1   shop_id       int64         
 2   item_id       int64         
 3   item_price    float64       
 4   item_cnt_day  float64       
dtypes: datetime64[ns](1), float64(2), int64(2)
memory usage: 134.4 MB


In [122]:
temp = sales_train.loc[(sales_train['item_cnt_day'] <= 0)]
temp 

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
2,05.01.2013,0,25,2552,899.0,-1.0
148,23.01.2013,0,25,2321,999.0,-1.0
175,07.01.2013,0,25,2199,1449.0,-1.0
807,02.01.2013,0,25,2330,599.0,-1.0
1041,13.01.2013,0,25,5034,1989.0,-1.0
...,...,...,...,...,...,...
2934243,26.10.2015,33,25,3917,449.0,-1.0
2934462,18.10.2015,33,25,4896,6398.0,-1.0
2935263,05.10.2015,33,25,10039,249.0,-1.0
2935643,16.10.2015,33,25,7893,2990.0,-1.0


In [123]:
temp.describe()

,date_block_num,shop_id,item_id,item_price,item_cnt_day
count,7356.00000,7356.000000,7356.000000,7356.000000,7356.000000
mean,14.63037,32.514546,8982.021207,1442.945623,-1.025150
std,9.43772,16.825725,6166.315366,2419.680132,0.368794
min,0.00000,2.000000,28.000000,0.500000,-22.000000
25%,7.00000,19.000000,3734.000000,349.000000,-1.000000
50%,13.00000,31.000000,6927.000000,799.000000,-1.000000
75%,23.00000,47.000000,14056.000000,1699.000000,-1.000000
max,33.00000,59.000000,22167.000000,33490.000000,-1.000000


In [120]:
sales_train.loc[(sales_train['item_cnt_day'] > 800)]

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
2326930,15.01.2015,24,12,20949,4.000000,1000.0
2909818,28.10.2015,33,12,11373,0.908714,2169.0


# Test #


In [169]:
test.head()

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


In [170]:
test.shape

(214200, 3)

In [33]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 214200 entries, 0 to 214199
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype
---  ------   --------------   -----
 0   ID       214200 non-null  int64
 1   shop_id  214200 non-null  int64
 2   item_id  214200 non-null  int64
dtypes: int64(3)
memory usage: 4.9 MB


In [35]:
test.loc[test.duplicated()]

,ID,shop_id,item_id


In [77]:
test.describe()

,ID,shop_id,item_id
count,214200.000000,214200.000000,214200.000000
mean,107099.500000,31.642857,11019.398627
std,61834.358168,17.561933,6252.644590
min,0.000000,2.000000,30.000000
25%,53549.750000,16.000000,5381.500000
50%,107099.500000,34.500000,11203.000000
75%,160649.250000,47.000000,16071.500000
max,214199.000000,59.000000,22167.000000
